In [ ]:
import pandas as pd
from io import BytesIO
from PIL import Image
import requests
import os

In [ ]:
df = pd.read_parquet("hf://datasets/bghira/photo-concept-bucket/photo-concept-bucket.parquet")

In [ ]:
# Confirm that the data is loaded and column of url exists
df.head(1)

In [ ]:
index = 0
# I want 200 images
output_dir = '../images/training_set/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)
img_num = 0
while len(os.listdir(output_dir)) < 300:
    url = df.iloc[index]['url']
    
    response = requests.get(url)
    image = None
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
    else:
        index += 1
        continue
    
    width, height = image.size
    
    # Keep only images >= 4k resolution
    if width < 3840 and height < 2160:
        index += 1
        continue

    # Cropping image to 3840x2160 by getting a box in the center of larger image
    # Center box on image
    target_width, target_height = 3840, 2160
    left = (width - target_width) // 2
    top = (height - target_height) // 2
    right = left + target_width
    bottom = top + target_height
    crop_box = (left, top, right, bottom)
    cropped_image = image.crop(crop_box)
    
    cropped_image.save(f'{output_dir}image_{img_num}.png')
    img_num += 1
    index += 1